In [1]:
import ee
import sys
sys.path.insert(1, "../apGIS/")
sys.path.insert(2, "/test")
import apgis
from apgis.apgeojson import GeoJSON
from apgis.apfield import Field
import apgis.geespatial as spatial
from apgis.apdate import Date
import apgis.apexception as apexception 

In [2]:
apgis.eeInitialize()

In [3]:
import geemap
Map = geemap.Map()
Map.add_basemap("SATELLITE")

In [41]:
def genSZRender(ndviLayerCol, ndvi: ee.Image, field: Field) -> ee.FeatureCollection:
    try:
        def applySNIC(clusterImage):

            seeds = ee.Algorithms.Image.Segmentation.seedGrid(size=round(threshPixel / 4), gridType='hex')
            geoClusterImage = (ee.Algorithms.Image.Segmentation.SNIC(image=clusterImage, size=round(threshPixel / 2),
                                                                     compactness=5, seeds=seeds).select(['clusters'])
                               .clip(clusterImage.geometry()))

            return geoClusterImage

        cleanSZ = spatial.genCleanSZ(ndviLayerCol, field)
        
        if cleanSZ.size().getInfo() == 0:
            raise apexception.EEEmptyCollectionError

        areaSZ = cleanSZ.map(spatial.setArea)

        threshArea = field.eeAOI.area(5).multiply(0.2).round().getInfo()
        threshPixel = round(threshArea / 100)

        bigSZLayers, smallSZLayers = spatial.splitSZLayers(areaSZ, threshArea)

        if bigSZLayers.size().getInfo() != 0: 
            clusterImageList = spatial.areaLayers(bigSZLayers)
            clusterCollection = ee.ImageCollection.fromImages(clusterImageList)

            geoClusters = clusterCollection.map(applySNIC)

            clusterFeatureList = ee.List(geoClusters.iterate(spatial.getClusterFeatureList, ee.List([]))).flatten()
            SNICCol = ee.FeatureCollection(clusterFeatureList).merge(smallSZLayers)    

        else:
            SNICCol = smallSZLayers

        SNICCol = ndvi.reduceRegions(collection=SNICCol, reducer=ee.Reducer.mean()).map(spatial.fixSNIC)
        SNICCol = SNICCol.map(spatial.setArea)      
        
        
        if SNICCol.size().getInfo() == 0:
            raise apexception.EEEmptyCollectionError
    
        return SNICCol

    except Exception as e:
        raise apexception.EERuntimeError(f"StressZone SNIC Render Generation Failed @ Runtime: {e}")

In [5]:
ndvi2Vis = {
    'min': 0, 
    'max': 10, 
    'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901', '66A000', '529400', 
                '3E8601', '207401', '056201', '004C00', '023B01', '012E01', '011D01', '011301']
}

S2TC = {
    'min': 0, 
    'max': 255,
    'bands': ['TCI_R', 'TCI_G', 'TCI_B']
}

In [6]:
# from glob import glob 
# # files = glob("s://antpod//testspatial//*.geojson")
# files = sorted(glob('*.geojson'))
# print(files)

# for index, file in enumerate(files):
#     localfileName = file.split('.')[0]
#     field = Field(localfile=f"{localfileName}.geojson")
#     print(spatial.getArea(ee.Geometry(field.geometry), "HA"))

In [26]:
localfileName = "f-15"

drange = (Date("2020-05-23"), Date("2020-09-30"))
field = Field(localfile=f"{localfileName}.geojson")

S2 = apgis.gee.genCollection(sensor="L2A", daterange=drange, geometry=field.eeROI)\
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15))

print(S2.size().getInfo())

8


In [27]:
image = S2.first().clip(field.eeROI)

In [28]:
Map.addLayer(field.eeAOI, {}, "bounds")

In [29]:
Map.addLayer(image, S2TC, "TC")
Map.centerObject(field.eeROI,zoom=15)

In [30]:
Map

Map(bottom=3939046.0, center=[10.89992300098001, 79.03842049993624], controls=(WidgetControl(options=['positio…

In [31]:
ndvi = apgis.index.generateIndex(image=image, sensor="L2A", index="NDVI").clip(field.eeAOI)
ndmi = apgis.index.generateIndex(image=image, sensor="L2A", index="NDMI").clip(spatial.genBufferGeo(field, 50))

In [32]:
Map.addLayer(ndvi, ndvi2Vis, "ndvi")

In [33]:
ndviRender = spatial.renderNDVI(ndvi)
ndmiRender = spatial.renderNDMI(ndmi)

ndviLayerRender = spatial.layerCoding(ndviRender)
ndmiLayerRender = spatial.layerCoding(ndmiRender)

ndviLayerCol = spatial.mergeLayerPolygons(ndviLayerRender)
ndmiLayerCol = spatial.mergeLayerPolygons(ndmiLayerRender)

indexes = ndvi.addBands(ndmi)

rawData = spatial.accumulateRawValue(indexes, field)
scoredData = spatial.scoreBuilder(rawData)

In [34]:
ndviLayerRender.size().getInfo() 

27

In [35]:
ndviLayerCol.size().getInfo()

7

In [84]:
def applySNIC(clusterImage):
    seeds = ee.Algorithms.Image.Segmentation.seedGrid(size=round(threshPixel / 5), gridType='hex')
    geoClusterImage = (ee.Algorithms.Image.Segmentation.SNIC(image=clusterImage, size=round(threshPixel / 2),
                                                             compactness=5, seeds=seeds).select(['clusters'])\
                       .clip(clusterImage.geometry()))

    return geoClusterImage

In [85]:
cleanSZ = spatial.genCleanSZ(ndviLayerCol, field)


areaSZ = cleanSZ.map(spatial.setArea)

threshArea = field.eeAOI.area(5).multiply(0.2).round().getInfo()
threshPixel = round(threshArea / 100)

bigSZLayers, smallSZLayers = spatial.splitSZLayers(areaSZ, threshArea)

if bigSZLayers.size().getInfo() != 0: 
    clusterImageList = spatial.areaLayers(bigSZLayers)
    clusterCollection = ee.ImageCollection.fromImages(clusterImageList)

    geoClusters = clusterCollection.map(applySNIC)

    clusterFeatureList = ee.List(geoClusters.iterate(spatial.getClusterFeatureList, ee.List([]))).flatten()
    SNICCol = ee.FeatureCollection(clusterFeatureList).merge(smallSZLayers)    

else:
    SNICCol = smallSZLayers

SNICCol = ndvi.reduceRegions(collection=SNICCol, reducer=ee.Reducer.mean()).map(spatial.fixSNIC)
SNICCol = SNICCol.map(spatial.setArea)      

In [86]:
geoClusters = clusterCollection.map(applySNIC)
geoClusters.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'clusters',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': -2147483648,
      'max': 2147483647},
     'dimensions': [1, 1],
     'origin': [78, 9],
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'system:footprint': {'type': 'Polygon',
     'coordinates': [[[78.74333539917394, 9.98375030356458],
       [78.74333539917394, 9.98374132041174],
       [78.7433264160211, 9.98374132041174],
       [78.7433264160211, 9.983732337258898],
       [78.74331743286825, 9.983732337258898],
       [78.74331743286825, 9.983723354106058],
       [78.74330844971541, 9.983723354106058],
       [78.74330844971541, 9.983714370953216],
       [78.74329946656258, 9.983714370953216],
       [78.74329946656258, 9.983705387800375],
       [78.74329048340974, 9.983705387800375],
       [78.74329048340974, 9.983687421494691],
       [78.7432815002569

In [42]:
SNICCol = genSZRender(ndviLayerCol, ndvi, field)

In [43]:
SNICCol.size().getInfo()

1

In [38]:
SNICCol = SNICCol.map(spatial.setArea)      

In [44]:
Map.addLayer(SNICCol, {}, "sz")

In [ ]:
resource.eeAOI.getInfo()

In [ ]:
sample = ee.FeatureCollection(ee.Feature(resource.eeAOI))
sample = sample.set("status", "no stress zones")

In [ ]:
sample.getInfo()

In [ ]:
task = ee.batch.Export.table.toCloudStorage(collection= sample, 
                                                bucket="antpod-apgis-exports", 
                                                fileNamePrefix="sample2",
                                                fileFormat= 'GeoJSON',
                                                selectors= ["status"])

In [ ]:
task.start()

In [ ]:
task.status()

In [ ]:
# geo =  GeoJSON(file=f"{localfileName}.geojson")
# farmData = geo.geoData

# field  = Field(geojson=geo)
# centroid = field.centroid
# bbox = field.ROIBox
# polybox = field.ROI
# farmBounds = field.AOI

# units = ["SQM", "SQKM", "HA", "ACRE"]
# areaData = {}

# for unit in units:
#     area = spatial.getArea(field.eeAOI, unit)
#     areaData[unit] = area
    
# meta = {"area":areaData, "centroid": centroid, "bbox": bbox, "polybox":polybox, "bounds": farmBounds}
# farmData['properties']["farmData"] = meta

In [ ]:
exportList = {
    "NDMIRender": ndviLayerCol, 
    "NDVIRender": ndmiLayerCol, 
    "RawData": rawData, 
    "SZRender": SNICCol,
    #"Farm": farmData
}

In [ ]:
tasklist = []
for exportName, exportObj in exportList.items():
    filePrefix = f"{localfileName}-2-TestExportAcq"
    task = ee.batch.Export.table.toCloudStorage(collection= exportObj, 
                                                bucket="antpod-apgis-exports", 
                                                fileNamePrefix=f"{filePrefix}-{exportName}",
                                                fileFormat= 'GeoJSON')
    tasklist.append(task)

In [ ]:
for task in tasklist:
    task.start()

In [ ]:
tasklist[3].status()